# Question 3: Trajectory Evaluation and g2o

_Refer to the example notebooks for installation instructions_

# Evo

So you've implemented 2D SLAM, great! Now, what? We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

## Write up with plots/images
**APE:** The absolute pose error is a metric for investigating the global consistency of a SLAM trajectory. APE is based on the absolute relative pose between two poses $P_{ref,i}, P_{est,i} \in \mathrm{SE}(3)$ at timestamp $i$:
\begin{equation*} E_i = P_{est,i} \ominus P_{ref,i} = P_{ref,i}^{-1} P_{est,i} \in \mathrm{SE}(3) \end{equation*}
where $\ominus$ is the inverse compositional operator, which takes two poses and gives the relative pose.
Different pose relations can be used to calculate APE like the translational part, rotation angle, rotational part or the full relative pose of $E_i$

On comparing APE of the ground truth poses kitti file with the poses that we obtained using 2D SLAM, we saw the following results:
```bash
--------------------------------------------------------------------------------
APE w.r.t. translation part (m)
(not aligned)

       max	6.761800
      mean	2.053709
    median	1.861706
       min	0.001024
      rmse	2.783288
       sse	929.602719
       std	1.878555

--------------------------------------------------------------------------------
```

![APE](../q3_imgs/ape_raw_25_250_500.png)

**RPE:** The relative pose error is a metric for investigating the local consistency of a SLAM trajectory. RPE compares the relative poses along the estimated and the reference trajectory. This is based on the delta pose difference: 
\begin{equation*} E_{i,j} = \delta_{est_{i,j}} \ominus \delta_{ref_{i,j}} = (P_{ref,i}^{-1}P_{ref,j})^{-1} (P_{est,i}^{-1}P_{est,j}) \in \mathrm{SE}(3) \end{equation*}
where $\ominus$ is the inverse compositional operator, which takes two poses and gives the relative pose.


On comparing RPE of the ground truth poses kitti file with the poses that we obtained using 2D SLAM, we saw the following results:
```bash
--------------------------------------------------------------------------------
RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(not aligned)

       max	0.296480
      mean	0.117295
    median	0.112402
       min	0.010395
      rmse	0.132775
       sse	2.097860
       std	0.062217

--------------------------------------------------------------------------------
```
![RPE](../q3_imgs/rpe_raw_25_250_500.png)

**Traj:**
We've used `evo_traj` to compare the ground truth trajectory with the optimised trajectory that we obtained using 2D SLAM. The info_values for 2D SLAM algo were [25,250,500]
```bash
--------------------------------------------------------------------------------
name:	gt
infos:
	nr. of poses	120
	path length (m)	52.97651922876868
	pos_end (m)	[-2.8000011 -4.5000024  0.       ]
	pos_start (m)	[-8.  5.  0.]
--------------------------------------------------------------------------------
name:	opt25_250_500
infos:
	nr. of poses	120
	path length (m)	53.75047013921479
	pos_end (m)	[-2.8360977 -1.5989153  0.       ]
	pos_start (m)	[-8.000984   4.9997168  0.       ]
```
Trajectories:
![trajectories](../q3_imgs/traj_traj_25_250_500.png)
XYZ:
![XYZ](../q3_imgs/traj_xyz_25_250_500.png)
Roll-Pitch-Yaw:
![RPY](../q3_imgs/traj_rpy_25_250_500.png)

As we can see from the XYZ graph, X axis shows the more error.

# g2o

Install g2o as mentioned in `examples/g2o.ipynb` and optimise `edges.txt`, the file you used earlier. Also use `g2o_viewer` and optimize `intel` (a trajectory in the Intel research lab) and `sphere`. They should look something like:


<table><tr>
<td> <img src="../misc/intel.jpg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="../misc/sphere.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

Write briefly about your observations and try out few options in the GUI. What do they do, how do they perform?

## Your answer
Using the g2o_viewer tool, we started out with:

![noisy initial trajectory](../q3_imgs/initial_traj.png)

Using LM algorithm on g2o_viewer, we finally obtained the following optimized trajectory which is very close to the ground truth:
![optimized trajectory](../q3_imgs/opt_traj.png)

Using the Guass Newton algorithm, we obtained similar results too.


Next we used the tool to find the trajectory for intel dataset:
![optimized trajectory](../q3_imgs/intel_lm_span.png)


Then we found the trajectory for sphere.g2o:
![optimized trajectory](../q3_imgs/sphere_lm_span.png)


We also tried varying the available options:

**Initial Guess:** 
For the given dataset, we compare the spanning tree and odometry initialization. The spanning-tree approach yields a guess close to the ground truth whereas odometry produces results with high errors which match with the ground truth to a certain extent. We’ve used spanning tree mostly when trying out the rest of the options.

**Optimizer:**
The variations of Gauss Newton and Levenberg Marquadt algorithms were found to produce similar kinds of results. So we have used gn_var_cholmod and lm_var_cholmod respectively for these two algorithms. For intel.g2o, we noticed that using GN algo, the graph converged within 10 iterations whereas LM took around 567 iterations. Also, the error for LM was higher than that for GN. We also noticed that using the 2d linear algorithm, we obtain very quick accurate results (in around 3 iterations) for the intel.g2o file, converging with errors similar to the GN algorithm. Such results are obtained because the optimizer is constructed for 2D pose-graph SLAM like the current one. 2d linear algorithm yields good performance on the intel.g2o.

**Robust Kernel:**
When results are obtained by applying the robust kernel option in g2o, the performance seems to be significantly improved as compared to the previously used approaches. A lower error value is obtained for some of the iterations. Among all the variations we tried out, the Huber kernel yielded the best results. Some of the other kernels resulted in low error values but differed from the ground truth. The Huber kernel required 147 iterations to converge ground truth, which is way less as compared to the results obtained without using kernel (459 iterations).
